In [1]:
#Importieren der Pakete für Solver, Investitionsentscheidung
from oemof.tools import economics
from oemof import solph
#Datenanalyse
import pandas as pd

In [2]:
#Einlesen der Datenbasis --- normierte Profile/Verbraucher/Einspeisungen/Erzeuger/Bezüge/Energieumwandler/Speicher
profile = pd.read_excel("input_data.xlsx", sheet_name = "norm_profile")
sink = pd.read_excel("input_data.xlsx", sheet_name = "sink")
source = pd.read_excel("input_data.xlsx", sheet_name = "source")
transformer = pd.read_excel("input_data.xlsx", sheet_name = "transformer")
storage = pd.read_excel("input_data.xlsx", sheet_name = "storage")

In [8]:
#Modelldefinition im Zeitraum 2045 in stdl. Auflösung
Zeit = pd.date_range("1/1/2045", periods=8760, freq="H")

#Modelldefinition
deSME = solph.EnergySystem(timeindex=Zeit)

#Gewichteter Kapitalkostensatz//Weighted Average Cost of Capital
r=0.05

#Auswahl der normierten Profile aus dem Excel-Datenblatt
cell_el = ["i0_el", "i1_el", "i2_el", "i4_el", "i5_el"]
cell_nt = ["nt1_th", "nt2_th"]
cell_ht = ["ht1_th", "ht2_th"]

for i in cell_el:
    if sink.profil.iloc[2] == i:
        lp_el = profile.loc[:,i]

for j in cell_nt:
    if sink.profil.iloc[3] == j:
        lp_nt = profile.loc[:,j]

for k in cell_ht:
    if sink.profil.iloc[4] == k:
        lp_ht = profile.loc[:,k]

#Definition der Energieträger bzw. Knotenpunkte als Bus
bel_sme = solph.Bus(label = "Strom KMU")
bel_pv_sme = solph.Bus(label = "Strom PV")
bel_chp_sme = solph.Bus(label = "Strom KWK")
bh2_sme = solph.Bus(label = "H2 KMU")
bbio_sme = solph.Bus(label = "Biomasse KMU")
bhps_sme = solph.Bus(label = "HD-Dampf KMU")
bth_h_sme = solph.Bus(label = "Prozesswaerme KMU")
bth_l_sme = solph.Bus(label = "Raumwaerme KMU")
btes_sme = solph.Bus(label = "WÜ-Verbindung KMU")

#Hinzufügen der Busses zum Energiesystem
deSME.add(bel_sme, bel_pv_sme, bel_chp_sme, bh2_sme, bbio_sme, bhps_sme, bth_h_sme, bth_l_sme, btes_sme)

###SENKEN -- SINKS
#Strom- und Wärmelastgänge sowie Abwärmesenken
el_cons_sme = solph.Sink(
    label="Stromlast KMU",
    inputs={bel_sme: solph.Flow(fix= lp_el, nominal_value= sink.bedarf.iloc[2])})

th_l_cons_sme = solph.Sink(
    label="NT-Waerme KMU",
    inputs={bth_l_sme: solph.Flow(fix= lp_nt, nominal_value= sink.bedarf.iloc[3])})

th_h_cons_sme = solph.Sink(
    label="HT-Waerme KMU",
    inputs={bth_h_sme: solph.Flow(fix= lp_ht, nominal_value = sink.bedarf.iloc[4])})

th_l_sink_sme = solph.Sink(
    label="NT-Wärmesenke",
    inputs={bth_l_sme: solph.Flow()})

th_h_sink_sme = solph.Sink(
    label="HT-Wärmesenke",
    inputs={bth_h_sme: solph.Flow()})

#Netzeinspeisungen Strom
pv_exp_sme = solph.Sink(
        label="Stromexport PV KMU",
        inputs={bel_pv_sme: solph.Flow(variable_costs = -1*sink.var_kosten.iloc[0])})


chp_exp_sme = solph.Sink(
        label="Stromexport KWK KMU",
        inputs={bel_chp_sme: solph.Flow(variable_costs = -1*sink.var_kosten.iloc[1])})

###QUELLEN -- SOURCES
#EE-Erzeugung
pv_sme = solph.Source(
        label="PV KMU",
        outputs={bel_pv_sme: solph.Flow(fix= profile.pv,
        investment = solph.Investment(
        ep_costs = economics.annuity(capex= source.invest.iloc[4], n=source.ANF.iloc[4], wacc=r),
        maximum = source.maximum.iloc[4]))})

#Netzbezüge Strom/Wasserstoff/Wärme/Biomasse
el_imp_sme = solph.Source(
        label="Stromimport Netz KMU",
        outputs={bel_sme: solph.Flow(variable_costs = source.var_kosten.iloc[2],
        investment = solph.Investment(ep_costs = source.invest.iloc[2]))})

h2_imp_sme = solph.Source(
        label="H2-Import KMU",
        outputs={bh2_sme: solph.Flow(variable_costs = source.var_kosten.iloc[0],
        investment = solph.Investment(ep_costs = source.invest.iloc[0]))})

th_imp_sme = solph.Source(
        label="Import Waermenetz KMU",
        outputs={bth_l_sme: solph.Flow(variable_costs = source.var_kosten.iloc[1],
        investment = solph.Investment(ep_costs = source.invest.iloc[1]))})

bio_imp_sme = solph.Source(
        label="Import Biomasse KMU",
        outputs={bbio_sme: solph.Flow(variable_costs = source.var_kosten.iloc[3],
        investment = solph.Investment(ep_costs = source.invest.iloc[3]))})


##ENERGIEWANDLER -- TRANSFORMER
wp_sme = solph.Transformer(
        label='Waermepumpe KMU',
        inputs={bel_sme: solph.Flow()},
        outputs={bth_l_sme: solph.Flow(
            investment=solph.Investment(
                ep_costs=economics.annuity(capex= transformer.invest.iloc[0], n= transformer.ANF.iloc[0], wacc= r),
                maximum = transformer.maximum.iloc[0],
                minimum = transformer.minimum.iloc[0]))},
            conversion_factors={bth_l_sme: transformer.η_th.iloc[0]})

elektrolyser_sme = solph.Transformer(
        label='Elektrolyseur KMU',
        inputs={bel_sme: solph.Flow()},
        outputs={bh2_sme: solph.Flow(
            investment=solph.Investment(
                ep_costs=economics.annuity(capex= transformer.invest.iloc[1], n= transformer.ANF.iloc[1], wacc= r),
                maximum = transformer.maximum.iloc[1],
                minimum = transformer.minimum.iloc[1])), bth_l_sme: solph.Flow()},
            conversion_factors={bh2_sme: transformer.η_h2.iloc[1], bth_l_sme: transformer.η_th.iloc[1]})

    #Gas- und Dampfkraftwerk
gt_chp_sme = solph.ExtractionTurbineCHP(
        label='H2-Turbine KMU',
        inputs={bh2_sme: solph.Flow()},
        outputs={bel_chp_sme: solph.Flow(
            investment=solph.Investment(
                ep_costs=economics.annuity(capex= transformer.invest.iloc[2], n= transformer.ANF.iloc[2], wacc= r),
                maximum = transformer.maximum.iloc[2],
                minimum = transformer.minimum.iloc[2])), bhps_sme: solph.Flow()},
            conversion_factors={bel_chp_sme: transformer.η_el.iloc[2], bhps_sme: transformer.η_th.iloc[2]},
            conversion_factor_full_condensation={bel_chp_sme: transformer.η_el_full.iloc[2]})

st_sme = solph.Transformer(
        label='Dampfturbine KMU', ##Dampfturbine zur Senkung des HT-Dampfes - Verstromung und Nutzung der NT-Wärme
        inputs={bhps_sme: solph.Flow()},
        outputs={bel_chp_sme: solph.Flow(
            investment=solph.Investment(
                ep_costs=economics.annuity(capex= transformer.invest.iloc[3],n= transformer.ANF.iloc[3], wacc= r),
                maximum = transformer.maximum.iloc[3],
                minimum = transformer.minimum.iloc[3])), bth_l_sme: solph.Flow()},
            conversion_factors={bel_chp_sme: transformer.η_el.iloc[3], bth_l_sme: transformer.η_th.iloc[3]})

pth_sme = solph.Transformer(
        label='Power-to-Heat KMU',
        inputs={bel_sme: solph.Flow()},
        outputs={bth_h_sme: solph.Flow(
            investment=solph.Investment(
                ep_costs=economics.annuity(capex= transformer.invest.iloc[4],n= transformer.ANF.iloc[4], wacc= r),
                maximum = transformer.maximum.iloc[4],
                minimum = transformer.minimum.iloc[4]))},
            conversion_factors={bth_h_sme: transformer.η_th.iloc[4]})

bio_boiler_sme = solph.Transformer(
        label='Bio-Heizkessel KMU',
        inputs={bbio_sme: solph.Flow()},
        outputs={bhps_sme: solph.Flow(
            investment=solph.Investment(
                ep_costs=economics.annuity(capex= transformer.invest.iloc[10],n= transformer.ANF.iloc[10], wacc= r),
                maximum = transformer.maximum.iloc[10],
                minimum = transformer.minimum.iloc[10]))},
            conversion_factors={bhps_sme: transformer.η_th.iloc[10]})

bz_sme = solph.Transformer(
        label='Brennstoffzelle KMU',
        inputs={bh2_sme: solph.Flow()},
        outputs={bel_chp_sme: solph.Flow(
            investment=solph.Investment(
                ep_costs=economics.annuity(capex= transformer.invest.iloc[5],n= transformer.ANF.iloc[5], wacc= r),
                maximum = transformer.maximum.iloc[5],
                minimum = transformer.minimum.iloc[5])), bth_l_sme: solph.Flow()}, 
            conversion_factors={bel_chp_sme: transformer.η_el.iloc[5], bth_l_sme: transformer.η_th.iloc[5]})
           

m_chp_sme = solph.Transformer(
        label='BHKW KMU',
        inputs={bh2_sme: solph.Flow()},
        outputs={bel_chp_sme: solph.Flow(
            investment=solph.Investment(
                ep_costs=economics.annuity(capex= transformer.invest.iloc[6],n= transformer.ANF.iloc[6], wacc= r),
                maximum = transformer.maximum.iloc[6],
                minimum = transformer.minimum.iloc[6])), bth_l_sme: solph.Flow()},
            conversion_factors={bel_chp_sme: transformer.η_el.iloc[6], bth_l_sme: transformer.η_th.iloc[6]})

bio_chp_sme = solph.Transformer(
        label='Bio-BHKW KMU',
        inputs={bbio_sme: solph.Flow()},
        outputs={bel_chp_sme: solph.Flow(
            investment=solph.Investment(
                ep_costs=economics.annuity(capex= transformer.invest.iloc[9],n= transformer.ANF.iloc[9], wacc= r),
                maximum = transformer.maximum.iloc[9],
                minimum = transformer.minimum.iloc[9])), bth_l_sme: solph.Flow()},
            conversion_factors={bel_chp_sme: transformer.η_el.iloc[9], bth_l_sme: transformer.η_th.iloc[9]})

##SPEICHER - Storages
el_stor_sme = solph.components.GenericStorage(
        label="Stromspeicher KMU",
        inputs={bel_sme: solph.Flow()},
        outputs={bel_sme: solph.Flow()},
            loss_rate= storage.loss_rate.iloc[0],
            initial_storage_level=0,
            inflow_conversion_factor= storage.η_in.iloc[0],
            outflow_conversion_factor= storage.η_out.iloc[0],
            investment=solph.Investment(
                ep_costs=economics.annuity(capex= storage.invest.iloc[0],n= storage.ANF.iloc[0], wacc= r)))

    #Wärmespeichersystem
th_l_stor_sme = solph.components.GenericStorage(
        label="Waermespeicher KMU",
        inputs={btes_sme: solph.Flow()},
        outputs={btes_sme: solph.Flow()},
            loss_rate= storage.loss_rate.iloc[1],
            initial_storage_level=0,
            inflow_conversion_factor= storage.η_in.iloc[1],
            outflow_conversion_factor= storage.η_out.iloc[1],
            investment=solph.Investment(
                ep_costs=economics.annuity(capex= storage.invest.iloc[1],n= storage.ANF.iloc[1], wacc= r)))

th_stor_in_sme = solph.Transformer(#Wärmeübertrager - Eingang
    label='WÜ KMU Ein',
    inputs={bth_l_sme: solph.Flow()},
    outputs={btes_sme: solph.Flow(
        investment=solph.Investment(
            ep_costs=economics.annuity(capex= transformer.invest.iloc[7],n= transformer.ANF.iloc[7], wacc= r)))},
        conversion_factors={btes_sme: transformer.η_th.iloc[7]})

th_stor_out_sme = solph.Transformer(#Wärmeübertrager - Ausgang
    label='WÜ KMU Aus',
    inputs={btes_sme: solph.Flow()},
    outputs={bth_l_sme: solph.Flow(
        investment=solph.Investment(
            ep_costs=economics.annuity(capex= transformer.invest.iloc[8],n= transformer.ANF.iloc[8], wacc= r)))},
        conversion_factors={bth_l_sme: transformer.η_th.iloc[8]})

h2_stor_sme = solph.components.GenericStorage(
    label="H2-Speicher KMU",
    inputs={bh2_sme: solph.Flow()},
    outputs={bh2_sme: solph.Flow()},
        loss_rate= storage.loss_rate.iloc[2],
        initial_storage_level=0,
        inflow_conversion_factor= storage.η_in.iloc[2],
        outflow_conversion_factor= storage.η_in.iloc[2],
        investment=solph.Investment(
            ep_costs=economics.annuity(capex= storage.invest.iloc[2],n= storage.ANF.iloc[2], wacc= r)))

#Verbindungen von Buses
pv_in_sme = solph.Transformer(
    label='PV Eigenverbrauch',
    inputs={bel_pv_sme: solph.Flow()},
    outputs={bel_sme: solph.Flow()},
        conversion_factors={bel_sme: 1})

chp_in_sme = solph.Transformer(
    label='KWK Eigenverbrauch',
    inputs={bel_chp_sme: solph.Flow()},
    outputs={bel_sme: solph.Flow()},
        conversion_factors={bel_sme: 1})

heat_exchange_sme = solph.Transformer(
    label='Wärmeübergabe Prozesswärme',
    inputs={bhps_sme: solph.Flow()},
    outputs={bth_h_sme: solph.Flow()},
        conversion_factors={bth_h_sme: 1})

#Hinzufügen aller Elemente zum Energiesystem
deSME.add(el_cons_sme,th_l_cons_sme,th_h_cons_sme,th_l_sink_sme,th_h_sink_sme,pv_exp_sme,chp_exp_sme,pv_sme,el_imp_sme,h2_imp_sme,bio_imp_sme,th_imp_sme,
              wp_sme,elektrolyser_sme,gt_chp_sme,st_sme,pth_sme,bz_sme,m_chp_sme,el_stor_sme,th_l_stor_sme,th_stor_in_sme,th_stor_out_sme,h2_stor_sme,
              pv_in_sme,chp_in_sme,heat_exchange_sme,bio_chp_sme,bio_boiler_sme)

#Lösen des Unternehmensmodells
om = solph.Model(deSME)

#Lineare Lösung über den Solver
om.solve(solver="cplex", solve_kwargs={"tee":True})
results = solph.processing.results(om)

#Ausgabe der Lastgänge verschiedener Sektoren
stromprof = solph.views.node(results, "Strom KMU")["sequences"]
pv_verkauf = solph.views.node(results, "Strom PV")["sequences"]
kwk_verkauf = solph.views.node(results, "Strom KWK")["sequences"]
waermeprof = solph.views.node(results, "Raumwaerme KMU")["sequences"]
th_h_prof = solph.views.node(results, "Prozesswaerme KMU")["sequences"]
h2_prof = solph.views.node(results, "H2 KMU")["sequences"]
hps_prof = solph.views.node(results, "HD-Dampf KMU")["sequences"]
bio_prof = solph.views.node(results, "Biomasse KMU")["sequences"]

#Ausgabe der investierten Leistungen und Kapazitäten im Modell
pv_invest = solph.views.node(results, "Strom PV")["scalars"]
el_invest = solph.views.node(results, "Strom KMU")["scalars"]
kwk_invest = solph.views.node(results, "Strom KWK")["scalars"]
h2_invest = solph.views.node(results, "H2 KMU")["scalars"]
th_l_invest = solph.views.node(results, "Raumwaerme KMU")["scalars"]
th_h_invest = solph.views.node(results, "Prozesswaerme KMU")["scalars"]
bat_invest = solph.views.node(results, "Stromspeicher KMU")["scalars"]
tes_invest = solph.views.node(results, "Waermespeicher KMU")["scalars"]
h2s_invest = solph.views.node(results, "H2-Speicher KMU")["scalars"]
bio_invest = solph.views.node(results, "Biomasse KMU")["scalars"]
              


Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 12.10.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2019.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\jonbaars.ID\AppData\Local\Temp\tmpdp6pe8aq.cplex.log' open.
CPLEX> Problem 'C:\Users\jonbaars.ID\AppData\Local\Temp\tmpjmnplrlg.pyomo.lp' read.
Read time = 0.94 sec. (46.37 ticks)
CPLEX> Problem name         : C:\Users\jonbaars.ID\AppData\Local\Temp\tmpjmnplrlg.pyomo.lp
Objective sense      : Minimize
Variables            :  455543  [Nneg: 455533,  Box: 10]
Objective nonzeros   :   52578
Linear constraints   :  446767  [Less: 166440,  Equal: 280327]
  Nonzeros           : 1222170
  RHS nonzeros       :   24683

Variables            : Min LB: 0.0000000        Max UB: 20000.00       
O